# 📘 Project Summary
This notebook loads employee and service task data from a JSON file and imports it into a Neo4j graph database using py2neo. Basically we combined __Field Service and Employee.__

## ✅ What we did:
- Created Department nodes and added all employees under their respective departments.

- Stored detailed Employee info including skills and tags.

- Created Task nodes for each service task (like a customer visit or job).

- Connected each Task to:

    - Its Customer.

    - The assigned Driver (linked to the Vehicle used).

    - All selected Technicians involved.

💡 This turns the data into a graph of relationships — making it easy to visualize who did what, when, and with whom!

In [8]:
import json
from py2neo import Graph, Node, Relationship, NodeMatcher

In [9]:
#Make Connection
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

In [10]:
# Load employee data from the JSON file
with open("data/employees.json", "r", encoding="utf-8") as file:
    emp_data = json.load(file)

In [11]:
# Get all department names
departments = emp_data["Employee"].keys()

# Create a node for each department
for dept_name in departments:
    # Check if the department node already exists to avoid duplicates
    existing_node = graph.nodes.match("Department", name=dept_name).first()
    if not existing_node:
        dept_node = Node("Department", name=dept_name)
        graph.create(dept_node)
        print(f"Created Department node: {dept_name}")
    else:
        print(f"Department node already exists: {dept_name}")

Created Department node: Administration
Created Department node: Unknown Department
Created Department node: Technician Team
Created Department node: Drivers
Created Department node: Customer Support Team
Created Department node: Human Resource Department


![All Departments](data/pics/departments.png)

In [12]:
def create_employee(dept_name, employee_index):
    employees = emp_data["Employee"].get(dept_name)
    if employees is None:
        print(f"❌ Department '{dept_name}' not found.")
        return

    if employee_index >= len(employees):
        print(f"❌ Index {employee_index} out of range for department '{dept_name}'.")
        return

    emp = employees[employee_index]
    skill_tags = emp.get("skill_tags", [])
    skills_list = emp.get("skills", [])

    # Create or find Employee
    emp_node = graph.nodes.match("Employee", id=emp.get("id")).first()
    if not emp_node:
        emp_node = Node(
            "Employee",
            id=emp.get("id"),
            name=emp.get("name") or "",
            gender=emp.get("gender") or "",
            contact=emp.get("contact") or "",
            email=emp.get("email") or "",
            private_address=emp.get("private_address") or "",
            job_title=emp.get("job_title") or "",
            working_shift=emp.get("working_shift") or "",
            skill_tags=skill_tags
        )
        graph.create(emp_node)
        print(f"✅ Created Employee: {emp.get('name')}")
    else:
        print(f"⚠️ Employee already exists: {emp.get('name')}")
        return

    # Link to Department
    dept_node = graph.nodes.match("Department", name=dept_name).first()
    if dept_node:
        graph.merge(Relationship(emp_node, "WORKS_IN", dept_node))
    else:
        print(f"❌ Department node not found: {dept_name}")

    # Create Skill nodes & relationships
    for skill in skills_list:
        skill_name = skill.get("name")
        progress = skill.get("progress")

        if skill_name:
            skill_node = graph.nodes.match("Skill", name=skill_name).first()
            if not skill_node:
                skill_node = Node("Skill", name=skill_name)
                graph.create(skill_node)

            rel = Relationship(emp_node, "HAS_SKILL", skill_node, progress=progress)
            graph.merge(rel)

In [13]:
def create_all_employees(dept_name):
    employees = emp_data["Employee"].get(dept_name)
    
    if not employees:
        print(f"❌ Department '{dept_name}' not found or has no employees.")
        return
    
    for idx in range(len(employees)):
        create_employee(dept_name, idx)

In [14]:
create_all_employees("Drivers")

✅ Created Employee: Safeer Ahmad Shah Muhammad
✅ Created Employee: Muhammad Asad Asghar Ali
✅ Created Employee: Waheed Ahmed Nisar Ahmed
✅ Created Employee: Ali Raza Muhammad Mehdi
✅ Created Employee: Muhammad Usman Khushi
✅ Created Employee: Zain Ul Hadi Noor Ul Hadi
✅ Created Employee: Nadeem
✅ Created Employee: Abu Hurraira
✅ Created Employee: Abdul Razzaq
✅ Created Employee: Adnan Yousaf
✅ Created Employee: Muhammad Asif
✅ Created Employee: Muhammad Nabeel
✅ Created Employee: Muhammad Shahid Muhammad Khalid
✅ Created Employee: Zeeshan Khalid Mehmod


![All Departments](data/pics/allDrivers.png)

In [15]:
create_employee("Technician Team", 5)
create_employee("Technician Team", 12)
create_employee("Technician Team", 13)
create_employee("Technician Team", 25)


✅ Created Employee: Marife Genotivo
✅ Created Employee: Ni Gusti Ayu Purnamawati
✅ Created Employee: Mellisa Mojares
✅ Created Employee: Rollyn Quitor Salvacion


![All Departments](data/pics/technicians.png)

# 2. Create Task

In [16]:
# Load employee data from the JSON file
with open("data/field_service.json", "r", encoding="utf-8") as file:
    task_data = json.load(file)

In [17]:
def create_task(task_items, graph):
    for item in task_items:
        # === 1. Create Task node ===
        task_data = item.get("service_task", {})
        planned_date = item.get("planned_date", {}).get("date")
        time_slot = item.get("time_slot", {})
        invoice = item.get("invoice", {})

        task = Node("Task",
                    id=task_data.get("id"),
                    name=task_data.get("standard_name"),
                    area=task_data.get("area"),
                    project=task_data.get("servis_name"),
                    planned_date=planned_date,
                    start_time=time_slot.get("start"),
                    end_time=time_slot.get("end"),
                    amount=invoice.get("amount"),
                    order_reference=invoice.get("order_reference"))
        graph.merge(task, "Task", "id")

        # === 2. Create Customer node ===
        customer_data = item.get("customer", {})
        if customer_data.get("id") is not None:
            customer = Node("Customer",
                            id=customer_data.get("id"),
                            name=customer_data.get("name"),
                            gender=customer_data.get("gender"),
                            contact=customer_data.get("contact"),
                            address=customer_data.get("address"),
                            email=customer_data.get("email"),
                            map_url=customer_data.get("map_url"))
            graph.merge(customer, "Customer", "id")
            graph.merge(Relationship(task, "HAS_CUSTOMER", customer))

        # === 3. Link Driver (as Employee) ===
        driver_data = item.get("driver", {})
        if driver_data.get("id") is not None:
            driver_node = graph.nodes.match("Employee", id=driver_data["id"]).first()
            if not driver_node:
                driver_node = Node("Employee",
                                   id=driver_data["id"],
                                   name=driver_data.get("name"))
                graph.create(driver_node)
            graph.merge(Relationship(task, "HAS_DRIVER", driver_node))

            # === 4. Vehicle + relationships ===
            vehicle_data = item.get("vehicle", {})
            if vehicle_data.get("id") is not None:
                vehicle = Node("Vehicle",
                               id=vehicle_data["id"],
                               name=vehicle_data.get("name"),
                               license_plate=vehicle_data.get("license_plate"))
                graph.merge(vehicle, "Vehicle", "id")
                graph.merge(Relationship(task, "USES_VEHICLE", vehicle))

        # === 5. Link Technicians (as Employees) ===
        selected_techs = item.get("technicians", {}).get("selected_technician", [])
        for tech in selected_techs:
            if tech.get("id") is not None:
                tech_node = graph.nodes.match("Employee", id=tech["id"]).first()
                if not tech_node:
                    tech_node = Node("Employee",
                                     id=tech["id"],
                                     name=tech.get("name"),
                                     resource_id=tech.get("resource_id"))
                    graph.create(tech_node)

                graph.merge(Relationship(task, "SELECTED_TECHNICIAN", tech_node))

In [18]:
#create few tasks only
create_task(task_data[6:11], graph)

![All Departments](data/pics/completeTask.png)